# Lesson 2: RAG Triad of metrics

In [30]:
import warnings
warnings.filterwarnings('ignore')

In [31]:
import utils

import os
import openai
openai.api_key = utils.get_openai_api_key()

In [32]:
from trulens_eval import Tru

tru = Tru()
tru.reset_database()

In [33]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["./eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

In [34]:
from llama_index import Document

document = Document(text="\n\n".\
                    join([doc.text for doc in documents]))

In [35]:
from utils import build_sentence_window_index

from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

sentence_index = build_sentence_window_index(
    document,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="sentence_index"
)

In [36]:
from utils import get_sentence_window_query_engine

sentence_window_engine = \
get_sentence_window_query_engine(sentence_index)

In [37]:
output = sentence_window_engine.query(
    "How do you create your AI portfolio?")
output.response

'You create your AI portfolio by showcasing a progression of projects that demonstrate your skills.'

## Feedback functions

In [38]:
import nest_asyncio

nest_asyncio.apply()

In [39]:
from trulens_eval import OpenAI as fOpenAI

provider = fOpenAI()

### 1. Answer Relevance

In [40]:
from trulens_eval import Feedback

f_qa_relevance = Feedback(
    provider.relevance_with_cot_reasons,
    name="Answer Relevance"
).on_input().on_output()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .


### 2. Context Relevance

In [41]:
from trulens_eval import TruLlama

context_selection = TruLlama.select_source_nodes().node.text

In [42]:
import numpy as np

f_qs_relevance = (
    Feedback(provider.qs_relevance,
             name="Context Relevance")
    .on_input()
    .on(context_selection)
    .aggregate(np.mean)
)

✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input statement will be set to __record__.app.query.rets.source_nodes[:].node.text .


In [43]:
import numpy as np

f_qs_relevance = (
    Feedback(provider.qs_relevance_with_cot_reasons,
             name="Context Relevance")
    .on_input()
    .on(context_selection)
    .aggregate(np.mean)
)

✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input statement will be set to __record__.app.query.rets.source_nodes[:].node.text .


### 3. Groundedness

In [44]:
from trulens_eval.feedback import Groundedness

grounded = Groundedness(groundedness_provider=provider)

In [45]:
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons,
             name="Groundedness"
            )
    .on(context_selection)
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


## Evaluation of the RAG application

In [46]:
from trulens_eval import TruLlama
from trulens_eval import FeedbackMode

tru_recorder = TruLlama(
    sentence_window_engine,
    app_id="App_1",
    feedbacks=[
        f_qa_relevance,
        f_qs_relevance,
        f_groundedness
    ]
)

In [47]:
eval_questions = []
with open('eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        eval_questions.append(item)

In [48]:
eval_questions

['What are the keys to building a career in AI?',
 'How can teamwork contribute to success in AI?',
 'What is the importance of networking in AI?',
 'What are some good habits to develop for a successful career?',
 'How can altruism be beneficial in building a career?',
 'What is imposter syndrome and how does it relate to AI?',
 'Who are some accomplished individuals who have experienced imposter syndrome?',
 'What is the first step to becoming good at AI?',
 'What are some common challenges in AI?',
 'Is it normal to find parts of AI challenging?']

In [49]:
eval_questions.append("How can I be successful in AI?")

In [50]:
eval_questions

['What are the keys to building a career in AI?',
 'How can teamwork contribute to success in AI?',
 'What is the importance of networking in AI?',
 'What are some good habits to develop for a successful career?',
 'How can altruism be beneficial in building a career?',
 'What is imposter syndrome and how does it relate to AI?',
 'Who are some accomplished individuals who have experienced imposter syndrome?',
 'What is the first step to becoming good at AI?',
 'What are some common challenges in AI?',
 'Is it normal to find parts of AI challenging?',
 'How can I be successful in AI?']

In [55]:
for question in eval_questions:
    with tru_recorder as recording:
        sentence_window_engine.query(question)

In [56]:
records, feedback = tru.get_records_and_feedback(app_ids=[])
records.head()

app_id  \
0  App_1   
1  App_1   
2  App_1   
3  App_1   
4  App_1   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [57]:
import pandas as pd

pd.set_option("display.max_colwidth", None)
records[["input", "output"] + feedback]

,input,output,Context Relevance,Answer Relevance,Groundedness
0,"""What are the keys to building a career in AI?""","""The keys to building a career in AI involve learning foundational technical skills, working on projects, finding a job, and being part of a supportive community.""",0.9,1.0,0.500000
1,"""How can teamwork contribute to success in AI?""","""Teammates play a crucial role in the success of AI projects. Working collaboratively with colleagues who are dedicated, continuously learning, and focused on using AI to benefit all people can significantly impact the outcome of a project. The influence of teammates can inspire individuals to work harder, learn more, and contribute effectively to the project's success.""",0.8,0.9,0.566667
2,"""What is the importance of networking in AI?""","""Networking in AI is crucial as it can provide valuable insights, guidance, and opportunities for individuals looking to advance in the field. By connecting with professionals who have experience in AI, individuals can gain knowledge about the industry, potential career paths, and current trends. Networking also allows individuals to build relationships with others in the field, which can lead to mentorship, collaboration on projects, and even job opportunities. Additionally, networking can help individuals stay updated on the latest developments in AI and expand their professional connections for future career growth.""",NaN,1.0,0.000000
3,"""What are some good habits to develop for a successful career?""","""Developing good habits in areas such as eating, exercise, sleep, personal relationships, work, learning, and self-care can help individuals move forward in their careers while maintaining their health.""",NaN,1.0,NaN
4,"""How can altruism be beneficial in building a career?""","""Helping others during one's career journey can lead to better outcomes for oneself.""",NaN,1.0,NaN
5,"""What is imposter syndrome and how does it relate to AI?""","""Imposter syndrome is a phenomenon where individuals doubt their accomplishments and have a persistent fear of being exposed as a fraud. In the context of AI, newcomers to the field may experience imposter syndrome, feeling like they do not truly belong in the AI community despite their success. This can be a significant challenge for individuals in AI, as it may hinder their growth and progress in the field.""",NaN,NaN,NaN
6,"""Who are some accomplished individuals who have experienced imposter syndrome?""","""Former Facebook COO Sheryl Sandberg, U.S. first lady Michelle Obama, actor Tom Hanks, and Atlassian co-CEO Mike Cannon-Brookes.""",NaN,NaN,NaN
7,"""What is the first step to becoming good at AI?""","""Identifying possible roles and conducting informational interviews.""",NaN,NaN,NaN
8,"""What are some common challenges in AI?""","""Common challenges in AI include the evolving nature of technologies, the need to stay updated with changing technology, the difficulty in finding suitable projects and estimating timelines, the iterative nature of AI projects leading to challenges in project management, the necessity to collaborate with stakeholders lacking AI expertise, and the ongoing challenge of reading and understanding complex research papers in the field.""",NaN,NaN,NaN
9,"""Is it normal to find parts of AI challenging?""","""It is normal to find parts of AI challenging.""",NaN,NaN,NaN


In [58]:
tru.get_leaderboard(app_ids=[])

,Context Relevance,Answer Relevance,Groundedness,latency,total_cost
app_id,,,,,
App_1,0.654545,0.927273,0.574242,1.090909,0.000839


In [59]:
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path: https://s172-29-24-123p38560.lab-aws-production.deeplearning.ai/


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>